In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
xlsx = pd.ExcelFile(r'C:\work\study\physics\aaaaaaaaaa暑期科研mpsd\2Dgraphene in a cavity\data\self-consistency\Matsubara_self.xlsx')
df = pd.read_excel(xlsx, 'Sheet1')
npoints=50     # same as nu*print_window in iritate.cpp      npoints should be less than 50
kpoints=160     #same as ku
r=int(npoints/2)
def filek(i):
    return np.asmatrix(df.loc[i*npoints:(i+1)*npoints-1].loc[:,['pn','Re[Sigma_aa]','Im[Sigma_aa]','Re[Sigma_bb]','Im[Sigma_bb]']].values)   # choose data with the i th k

p_aa = np.zeros((r,kpoints),dtype = complex)
q_aa = np.zeros((r,kpoints),dtype = complex)

p_bb = np.zeros((r,kpoints),dtype = complex)
q_bb = np.zeros((r,kpoints),dtype = complex)

for k in range(0,kpoints):
    
    print('solving k at the ',k,' th points')
    
    # full Pade matrix can be separated into two parts under the assumption of diagnized self-senergy matrix
    a_aa = np.zeros((2*r,2*r),dtype = complex)
    a_bb = np.zeros((2*r,2*r),dtype = complex)
    for i in range (0,2*r):
        for jj in range(0,r):
            a_aa[i,jj]=(filek(k)[i,0]*1j)**jj               #the machine error in this step is large due to the power of r
            a_bb[i,jj]=a_aa[i,jj]                # Identity 2 by 2 operator
        for jj in range (r,2*r):
            a_aa[i,jj]=-(filek(k)[i,1]+1j*filek(k)[i,2])*(filek(k)[i,0]*1j)**(jj-r)      #the machine error in this step is large due to the power of r
            a_bb[i,jj]=-(filek(k)[i,3]+1j*filek(k)[i,4])*(filek(k)[i,0]*1j)**(jj-r)     

    b_aa = np.zeros((2*r,1),dtype = complex)
    b_bb = np.zeros((2*r,1),dtype = complex)
    
    for i in range (0,2*r):
        b_aa[i,0]=(filek(k)[i,1]+1j*filek(k)[i,2])*(filek(k)[i,0]*1j)**r        #the machine error in this step is large due to the power of r
        b_bb[i,0]=(filek(k)[i,3]+1j*filek(k)[i,4])*(filek(k)[i,0]*1j)**r

    for i in range(0,r):
        p_aa[i,k]=np.dot(np.asmatrix(a_aa).I,b_aa)[i,0]
        q_aa[i,k]=np.dot(np.asmatrix(a_aa).I,b_aa)[i+r,0]
        p_bb[i,k]=np.dot(np.asmatrix(a_bb).I,b_bb)[i,0]
        q_bb[i,k]=np.dot(np.asmatrix(a_bb).I,b_bb)[i+r,0]
        
    print('finish k at the ',k,' th points')


def Sig_aa(k,w):  
    pp=0+0j
    qq=w**r+0j
    for i in range(0,r):
        pp+=p_aa[i,k]*(w)**i
        qq+=q_aa[i,k]*(w)**i
    return pp/qq

def Sig_bb(k,w):  
    pp=0+0j
    qq=w**r+0j
    for i in range(0,r):
        pp+=p_bb[i,k]*(w)**i
        qq+=q_bb[i,k]*(w)**i
    return pp/qq

#output examination of pade

xlsx = pd.ExcelFile(r'C:\work\study\physics\aaaaaaaaaa暑期科研mpsd\2Dgraphene in a cavity\data\self-consistency\Matsubara_self.xlsx')
df = pd.read_excel(xlsx, 'Sheet1')
kvalue=df.pop('k')
pnvalue=df.pop('pn')

k_order= np.zeros((npoints*kpoints,1),dtype = int)
for i in range(0,kpoints):
    for l in range(i*npoints,(i+1)*npoints):
        k_order[l,0]=i

Re_aa_using_pede= np.zeros((npoints*kpoints,1))
for order in range(0,kpoints*npoints):
    Re_aa_using_pede[order,0]=Sig_aa(k_order[order,0],pnvalue[order]*1j).real

Re_bb_using_pede= np.zeros((npoints*kpoints,1))
for order in range(0,kpoints*npoints):
    Re_bb_using_pede[order,0]=Sig_bb(k_order[order,0],pnvalue[order]*1j).real

saavalue=df.pop('Re[Sigma_aa]')
outdf = pd.DataFrame()
outdf.insert(0,'k',kvalue)
outdf.insert(1,'pn',pnvalue)
outdf.insert(2,'Re[Sigma_aa]',saavalue)
outdf.insert(3,'Re[Sigaa]_using_Pede',Re_aa_using_pede)
outdf.insert(4,'Re[Sigbb]_using_Pede',Re_bb_using_pede)
outdf.insert(5,'k_order',k_order)
outdf.to_excel(r'C:\work\study\physics\aaaaaaaaaa暑期科研mpsd\2Dgraphene in a cavity\data\self-consistency\pede.xlsx', sheet_name='Sheet1')
print('finish output pade examination ***************************************')

#output arpes
vf=4.2
et=0.001
def INVG0(k,w,dtype=complex):
    return np.matrix([[w,-k*vf],[-k*vf,w]])
def Retarded_Self(k_order,w,dtype=complex):
    return np.matrix([[Sig_aa(k_order,w),0],[0,Sig_bb(k_order,w)]])
def arpes(k,k_order,w):
    return (-1./(np.pi))*(np.trace((-Retarded_Self(k_order,w+1j*et)+INVG0(k,w+1j*et)).I)).imag
def arpes_bare(k,w):
    return (-1./(np.pi))*(np.trace(INVG0(k,w+1j*et).I)).imag


xlsx = pd.ExcelFile(r'C:\work\study\physics\aaaaaaaaaa暑期科研mpsd\2Dgraphene in a cavity\data\self-consistency\Matsubara_self.xlsx')
df = pd.read_excel(xlsx, 'Sheet1')
kvalue0=df.pop('k')

morepoints=9       #draw more points for each k, total number of points of k = kpoints, total number of points of w (per k point) =npoints*(1+morepoints)
kvalue=kvalue0
for i in range(0,morepoints):
    kvalue=kvalue.append(kvalue0)

wvalue=[1*np.random.random()-0.5 for i in range(0,npoints*kpoints*(1+morepoints))]

k_order= np.zeros((npoints*kpoints*(1+morepoints),1),dtype = int)
for i in range(0,kpoints):
    for l in range(i*npoints,(i+1)*npoints):
        k_order[l,0]=i
        for m in range(0,morepoints+1):
            k_order[l+m*npoints*kpoints,0]=i


ARPES= np.zeros((npoints*kpoints*(1+morepoints),1))
for order in range(0,kpoints*npoints*(1+morepoints)):
    ARPES[order,0]=arpes(np.array(kvalue)[order],k_order[order,0],wvalue[order])
    
ARPES_BARE= np.zeros((npoints*kpoints*(1+morepoints),1))
for order in range(0,kpoints*npoints*(1+morepoints)):
    ARPES_BARE[order,0]=arpes_bare(np.array(kvalue)[order],wvalue[order])


outdf = pd.DataFrame()
outdf.insert(0,'k',kvalue)
outdf.insert(1,'w',wvalue)
outdf.insert(2,'arpes',ARPES)
outdf.insert(3,'arpes_bare',ARPES_BARE)
outdf.insert(4,'k_order',k_order)
outdf.to_excel(r'C:\work\study\physics\aaaaaaaaaa暑期科研mpsd\2Dgraphene in a cavity\data\self-consistency\arpes.xlsx', sheet_name='Sheet1')
print('finish output arpes ***************************************')
